### 
![LOGO_ADWAY](http://www.adway-conseil.com/wp-content/themes/adway/images/logo_adway.png?raw=true "Logo Title Text 1")
<h1><span style="color:#006600"><div style="text-align:center;"> Formation Natural Language Processing </div></span></h1>

<h1><span style="color:#006600"><div style="text-align:center;"> Session 2 - Cas pratique  </div></span></h1>


<h2><div style="text-align:center;"> Quora Pair Question </div></h2>








**Prérequis** :  Avoir bien écouté la première session !




## Sommaire

Surprise !!!



## Introduction

L'objectif de cette compétition est d'identifier quelles questions posées sur Quora, un forum accueillant plus de 100 millions de visiteurs par mois, sont des doubles de questions déjà posées. 
Une telle étude peut être utile afin de fournir instantanément des réponses aux questions précédemment résolues. 
Cette compétition consiste donc à prédire si une paire de questions sur Quora est identique. Pour ce problème, nous avons environ 400.000 exemples d'entraînement. Chaque ligne se compose de deux phrases et d'une étiquette binaire qui nous indique si les deux questions étaient identiques ou non


### Description de la compétition
Le but de la compétition "Quora Question Pair" est de prédire laquelle des paires de questions fournies contient en réalité deux questions ayant le même sens.  


# <span style="color:#006600"> 1. Initialisation de l'environnement </span>

## <span style="color:#39ac39"> 1.1. Téléchargement des librairies </span>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import nltk.data
import re
import time
#from pattern.text.en import singularize
#from inflection import singularize
from nltk.corpus import stopwords
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
%matplotlib inline
#import heapq
import string 
import operator
import unicodedata
# import mathtqdm
import gensim
import itertools
import sys

import time # pour le timer
#import hunspell
#from spellchecker import SpellChecker
# from tqdm import tqdm_notebook, tqdm
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
    
    
#tqdm_notebook(tqdm.pandas())
pal = sns.color_palette()
#pd.options.display.max_colwidth=100
from functools import reduce

#Telecharger NLTK, wordnet

## <span style="color:#39ac39"> 1.2. Téléchargement des bases de données de train et test

La base de données d'apprentissage est composée des données suivantes : 
* **id** - identifiant de la paire de question
* **qid1, qid2** - respectivement, identifiant unique de la question 1 (resp. question 2 ) 
* **question1, question2** - texte intégrale pour chacune des questions composants la paire
* **is_duplicate** - il s'agit de la variable (variable a prédire) qui prend la valeur 1 si les deux questions ont le même sens, 0 sinon

In [ ]:
# Téléchargement de la base de train 
df_train = pd.read_csv('../input/quora-question-pairs/train.csv')

# Téléchargement de la base de test
df_test = pd.read_csv('../input/quora-question-pairs/test.csv', 
                      nrows=580000)


# <span style="color:#006600">2. Description des bases de données </span>

## <span style="color:#39ac39"> 2.1. Snapshot du train

In [ ]:
print("Dimension de la base de train : {} \n".format(df_train.shape)) # (404287, 5)
df_train.info()

In [ ]:
df_train.head(10)

## <span style="color:#39ac39"> 2.2. Snapshot du test

In [ ]:
print("Dimension de la base de test : {} \n".format(df_test.shape)) # (2345796, 3)
df_test.info()

In [ ]:
df_test.head(10)

## <span style="color:#39ac39"> 2.3. Quelques statistiques sur la base de train

In [ ]:
print("Analyse de la variable cible (is_duplicate) : \n******************************************** ")
print('Total number of question pairs for training: {}'.format(len(df_train)))
print('Duplicate pairs: {}% \n******************************************** '.format(round(df_train['is_duplicate'].mean()*100, 2)))
print('An example of duplicate pair : \nQuestion1 : {} \nQuestion2 : {} \n******************************************** '.format(df_train[df_train['is_duplicate']==1][1:2].question1.values,
                                                                               df_train[df_train['is_duplicate']==1][1:2].question2.values))
df_train.groupby("is_duplicate")['id'].count().plot.bar()


In [ ]:
# Suppression de la colonne ID et des valeurs manquantes 
df_train.drop(['id'], axis=1, inplace=True)
df_train = df_train.dropna()

# Concatenation des questions en une seule liste
qids = pd.Series(df_train['qid1'].tolist() + df_train['qid2'].tolist())

print('Total number of questions in the training data: {}'.format(len(np.unique(qids))))
print('Number of questions that appear multiple times: {}'.format(np.sum(qids.value_counts() > 1)))

plt.figure(figsize=(12, 5))
plt.hist(qids.value_counts(), bins=50)
plt.yscale('log', nonposy='clip')
plt.title('Log-Histogram of question appearance counts')
plt.xlabel('Number of occurences of question')
plt.ylabel('Number of questions')
plt.show()

## <span style="color:#39ac39"> 2.4. Datavisualisation - WordCloud

Un <span style="background-color: #ccffcc;"> wordcloud </span> (ou nuage de mots-clés / nuage de tags) est une répresentation visuelle de mots dont la taille est proportionelle à la fréquence de ce mot dans un texte donné.

In [ ]:
train_qs = pd.Series(df_train['question1'].tolist() + df_train['question2'].tolist()).astype(str)

from wordcloud import WordCloud
cloud = WordCloud(width=1440, 
                  height=1080).generate(" ".join(train_qs.astype(str)))
plt.figure(figsize=(20, 15))
plt.imshow(cloud)
plt.axis('off')

**Conclusion** : avec cette représentation graphique, on constate qu'il y a beaucoup de mot qui ne sont pas particulièrement utiles pour mettre en valeurs les grands thèmes des questions

# <span style="color:#006600">3. Prétraitement de la base et construction des features</span>

##  <span style="color:#39ac39"> 3.1. Suppression des phrases trop courtes
Ce premier prétraitement consiste à supprimer de notre base les phrases trop courte (moins de 11 caractères) car il est peu probable d'apprendre beaucoup de ces données. 

Quelques exemples sont affichés ci dessous : 

In [ ]:
print("Nombre de question 1 contenant moins de 11 caractères : ", len(df_train.index[df_train['question1'].apply(len)<11]))
print("Nombre de question 2 contenant moins de 11 caractères : ", len(df_train.index[df_train['question2'].apply(len)<11]))

print("\nAffichage de quelques exemples pour les questions 1 : \n -------------------------------------------------------- ")
listIndex1 = df_train.index[df_train['question1'].apply(len)<11][0:10]
print(df_train.loc[listIndex1].question1)

print("\nAffichage de quelques exemples pour les questions 2 : \n --------------------------------------------------------")
listIndex2 = df_train.index[df_train['question2'].apply(len)<11][0:10]
print(df_train.loc[listIndex2].question2)

Ces questions sont donc supprimées : 

In [ ]:
df_train.drop(df_train.index[df_train['question1'].apply(len)<11],inplace=True)
df_train.drop(df_train.index[df_train['question2'].apply(len)<11],inplace=True)

print("Dimension de la base de train : {}".format(df_train.shape)) # (404287, 5)
print("Dimension de la base de test : {}".format(df_test.shape)) # (2345796, 3)

 ## <span style="color:#39ac39"> 3.2. Creation d'une table id/question pour travailler sur le prétaitement de chaque phrase

### Sur la table d'apprentissage

** Démarche ** : on va créer une liste unique pour lancer les traitements sur toutes les observations en même temps

In [ ]:
# >>> Sur la base de train 
listdf = [pd.DataFrame(df_train[['qid1','question1']].values),
          pd.DataFrame(df_train[['qid2','question2']].values)]
qdf=pd.concat(listdf)
del listdf; gc.collect()
print("Dimension de notre liste de question : ", qdf.shape)
qdf.columns = ["id","question_orig"]

Verification s'il y a des id qui réapparaissent : 

In [ ]:
import collections
a = qdf.id.tolist()
res= [item for item, count in collections.Counter(a).items() if count > 1]
print("Liste des id qui sont présents plusieurs fois dans la base : ", list(res[0:10]))
print("\nAffichage d'un exemple : ") 
qdf[qdf.id == 3 ]

In [ ]:
# >>> Mise en forme de notre dataframe            
print("Nombre d'observation avant suppression des doublons : ", qdf.shape )
qdf.drop_duplicates(subset="id",inplace=True)
print("Nombre d'observation après suppression des doublons : ", qdf.shape )
qdf.reset_index(inplace=True)

# >>> Affichage d'un exemple
ni = 10000
print("Affichage de la question {} : \n {}".format(ni, list(qdf[qdf['id']==ni].question_orig)))

### Sur la table de test

In [ ]:
"""
# >>> Generation des index pour chacune des questions
a = pd.Series(range(df_test.shape[0]*2))+1
b = a.values.reshape(df_test.shape[0],2)
c = pd.DataFrame(b,
                 columns= ["qid1","qid2"],
                dtype='int64')

df_test['qid1']=np.nan
df_test['qid1'] = c.qid1

df_test['qid2']=np.nan
df_test['qid2'] = c.qid2

# >>> Creation d'une seule et unique liste, qui contient la liste de question1 puis la liste de question2 
# >>> Sur la base de train 
listdf = [pd.DataFrame(df_test[['qid1','question1']].values),
          pd.DataFrame(df_test[['qid2','question2']].values)]
qdf_test=pd.concat(listdf)
del listdf; gc.collect()
print("Dimension de notre liste de question : ", qdf_test.shape)

# >>> Mise en forme de notre dataframe
qdf_test.columns = ["id","question_orig"]
qdf_test.drop_duplicates(subset="id",inplace=True)
qdf_test.reset_index(inplace=True)

qdf_test.question_orig = qdf_test.question_orig.astype("str")

# >>> Affichage d'un exemple
ni = 10000
print("Affichage de la question {} : \n {}".format(ni, list(qdf_test[qdf_test['id']==ni].question_orig)))
"""

In [ ]:
def Trouve_Exemples(s):
    print(qdf['question_orig'][qdf["question_orig"].apply(lambda x : s in x)][0:10])
    
Trouve_Exemples("milk")
Trouve_Exemples("000")

 ## <span style="color:#39ac39"> 3.3. Creations de features basiques
 
 Communément, des features assez basiques sont créées. Ces features ont pour objectif de mettre en exergue les éléments les plus basiques d'une phrase telle que :
 * La **présence d'une lettre capitale** en début de phrase (méthode _isupper()_ )
 * La **présence d'un caractère** en particulier (ici, on va rechercher le point d'interrogation "?" ou le point ".")
 * La **présence de chiffre/nombre** (méthode _isdigit()_ )
 

In [ ]:
def basic_features(df):
    qmarks = np.mean(df["question_orig"].apply(lambda x: '?' in x))
    math = np.mean(df["question_orig"].apply(lambda x: '[math]' in x))
    fullstop = np.mean(df["question_orig"].apply(lambda x: '.' in x))
    capital_first = np.mean(df["question_orig"].apply(lambda x: x[0].isupper()))
    capitals = np.mean(df["question_orig"].apply(lambda x: max([y.isupper() for y in x])))
    numbers = np.mean(df["question_orig"].apply(lambda x: max([y.isdigit() for y in x])))

    print('Questions with question marks: {:.2f}%'.format(qmarks * 100))
    print('Questions with [math] tags: {:.2f}%'.format(math * 100))
    print('Questions with full stops: {:.2f}%'.format(fullstop * 100))
    print('Questions with capitalised first letters: {:.2f}%'.format(capital_first * 100))
    print('Questions with capital letters: {:.2f}%'.format(capitals * 100))
    print('Questions with numbers: {:.2f}%'.format(numbers * 100))


In [ ]:
# Lancement sur la base d'apprentissage
basic_features(qdf)

In [ ]:
"""
# Lancement sur la base de test
basic_features(qdf_test)
"""

##  <span style="color:#39ac39"> 3.4. Retraitement des phrases


**Objectif** : 
1. extraire les nombres et les formules de maths
2. réduire le vocabulaire en remplaçant les abréviations, en corrigeant les fautes d'orthographe et en lemmatizant
3. repérer le mot interrogatif
4. detecter les questions avec plusieurs phrases


### <span style="color:#53c653"> 3.4.1. Détection des équations
On va extraire tout ce qui est mathématiques (ce qui inclut les nombres). Une équation, c'est aussi simple et beau que ca : 

\begin{align}
\nabla \times \vec{\mathbf{B}} -\, \frac1c\, \frac{\partial\vec{\mathbf{E}}}{\partial t} & = \frac{4\pi}{c}\vec{\mathbf{j}} \\   \nabla \cdot \vec{\mathbf{E}} & = 4 \pi \rho \\
\nabla \times \vec{\mathbf{E}}\, +\, \frac1c\, \frac{\partial\vec{\mathbf{B}}}{\partial t} & = \vec{\mathbf{0}} \\
\nabla \cdot \vec{\mathbf{B}} & = 0
\end{align}

*Vous aurez deviner tous les équations de Maxwell ... *

Aussi belle que puisse être une équation manuscrite, l'écrire sur un éditeur de texte nécessite de nombreux tour de passe passe. 

Par exemple, et en utilisant Markdown comme langage  : 

\begin{align} \nabla \times \vec{\mathbf{B}} \end{align} s'écrit : \ nabla \ times \ vec{ \ mathbf { B } }


Sur Quora, on peut écrire des formules de maths à l'aide de la balise <span style="color:blue">**[/math]**</span> malheureusement tout le monde ne l'utilise pas...

Pour cela, nous allons utiliser plusieurs fonctions qui vont permettre d'identifier les formules contenues dans le texte : 
* **corrections_maths(s)** : harmonisation des balises, transcodification des abréviations et split les nombres avec les unités monétaires
* **Extrait_Math_Bien_Balise(s)** : sortir du texte les formules qui ont été correctement balisés. 
* **Extrait_Formule_Non_Balisee(s)** : Cette fonction détecter une section de phrase qui n'emploie que les termes généralement utilisés pour écrire des fonctions mathématiques (tel que x, y, z, 0, ... 9, +, -, cos...). Elle fait elle-même appel à deux autres fonctions :
  * **Verifie_Formule** qui fait un peu plus de tests (une formule doit au moins contenir un opérateur mathématique) et 
  * **Nombre(s)** (qui détermine si la formule n'est qu'un nombre) 

In [ ]:
def init_feature(df):
    df['question']=np.nan
    df['formule']=np.nan
    df['nombre']=np.nan
    df['questions_multiples']=np.nan
    df['n_questions']=np.nan
    df['mots_interrogatifs']=np.nan
    df['sac_de_mots']=np.nan

In [ ]:
# Initialisation des nouvelles variables sur la base d'apprentissage
init_feature(qdf)

In [ ]:
"""
# Initialisation des nouvelles variables sur la base d'apprentissage
init_feature(qdf_test)
"""

#### >>> Definition des fonctions qui nous permettent de réaliser la détection d'équation

In [ ]:
def Corrections_Maths(s):
    # Recuperation des balises maths
    s=s.replace('{/math]','[/math]')
    s=s.replace('[\math]','[/math]')
    
    # 60k -> 60000
    s=re.sub(r'(\d+)(k)(?=[^a-zA-Z])',r'\g<1>000',s)
    s=re.sub(r'(\d+)(K)(?=[^a-zA-Z])',r'\g<1>000',s)
    
    #Separer la monnaie
    s=s.replace('$',' $ ')
    s=s.replace('¥',' ¥')
    s=s.replace('€',' € ')
    s=s.replace('£',' £')
    
    return s

def Extrait_Math_Bien_Balise(s):
    formule = []
    reste_de_la_phrase = s
    
    e1=s.find("[math]")
    e2=s.find("[/math]")
    while(e1!=-1 and e2!=-1 and e1<e2):
        formule.append(reste_de_la_phrase[e1+6:e2])
        reste_de_la_phrase=reste_de_la_phrase[0][:e1] + reste_de_la_phrase[0][e2+7:]
        e1=reste_de_la_phrase.find("[math]")
        e2=reste_de_la_phrase.find("[/math]")
    return pd.Series([reste_de_la_phrase, formule])

def Nombre(s):
    s=s.replace(',','')
    try:
        float(s)
        return float(s)
    except ValueError:
        return -0.135792468

def Verifie_Formule(s):
    if len(s)==0:
        return False
    #print(s)
    if s[0]!=' ':
        #print('cond1')
        return False
    if re.search(r'[^ ∝∞√%\.,\|:!\-/×=<>\^\+\*\(\)\[\]\{\}…]',s)==None:
        #print('cond2')
        return False
    if re.search(r'[0-9∝∞√%\.,:!\|\-/×=<>\^\+\*\(\)\[\]\{\}…]',s)==None:
        #print('cond3')
        return False
    return True


#Pas bon, cf exemples#
def Extrait_Formule_Non_Balisee(s):
    
    reste_de_la_phrase = s
    nombre = []
    formule = []
    
    formules = re.findall(r' (?:[0-9xyz\|\(\[{]|cos |cos\(|sin |sin\(|tan |tan\(|csc |csc\(|exp |exp\(|log |log\(|ln||alpha|beta|gamma|delta|theta|pi ])(?:[0-9∝∞√%\|\.×,:!\-/=<> \^\+\*\(\)\[\]\{\}…xyz]|cos |cos\(|sin |sin\(|tan |tan\(|csc |csc\(|exp |exp\(|log |log\(|ln|alpha|beta|gamma|delta|theta|pi)*',s)#(?=[^\.:,\(\[\{])',s)#(?:[0-9xyz!)\]}]|alpha|beta|gamma|delta|theta|pi])(?=[^a-zA-Z]|$)',s)#*',s)
    #print(formules)
    formules = filter(Verifie_Formule,formules)
    #print(formules)
    for x in formules:
        x=x.strip()
        l = Nombre(x)
        if(l!= -0.135792468):
            nombre.append(l)
        else:
            formule.append(x)
        e1=reste_de_la_phrase.find(x)
        reste_de_la_phrase=reste_de_la_phrase[:e1]+" "+reste_de_la_phrase[e1+len(x):]
        
    nombre.sort()
    
    return pd.Series([reste_de_la_phrase, nombre, formule])



Lancement des fonctions précédemment définies pour identifier les équations dans le texte.

#### >>> Lancement sur la base de train : 

In [ ]:
# Creation d'une nouvelle variable "question" qui subira les différentes transformations
qdf['question']=qdf['question_orig']

In [ ]:
import time

start = time.time()
print("Lancement des traitements mathématiques : \n")

print("   >>> Corrections simples...")
qdf['question']=qdf['question'].apply(Corrections_Maths)

print("   >>> Extractions simples...")
qdf[['question','formule']] = qdf['question'].apply(Extrait_Math_Bien_Balise)

print("   >>> Extractions compliquees...")
qdf[['question','nombre','formule']] = qdf['question'].apply(Extrait_Formule_Non_Balisee)

print("\nFin des traitements mathématiques : \n")
print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

#### >>> Lancement sur la base de test : 

Dans l'idéal, chacune des étapes qui vont suivre doivent etre lancé sur la base de train et la base de test pour pouvoir réaliser les prédictions sur cette dernière 

In [ ]:
"""

# Initialisation des nouvelles variables sur la base de test
start = time.time()
print("Lancement des traitements mathématiques : \n")

print("   >>> Corrections simples...")
qdf_test['question']=qdf_test['question_orig'].apply(Corrections_Maths)

print("   >>> Extractions simples...")
qdf_test[['question','formule']] = qdf_test['question'].apply(Extrait_Math_Bien_Balise)

print("   >>> Extractions compliquees...")
qdf_test[['question','nombre','formule']] = qdf_test['question'].apply(Extrait_Formule_Non_Balisee)

print("\nFin des traitements mathématiques : \n")
print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

"""

#### >>> Quelques affichages 

In [ ]:
print(qdf[['question_orig','formule']][qdf['formule'].apply(len)>0][0:10])
print(qdf[['question_orig','nombre']][qdf['nombre'].apply(len)>0][0:10])

In [ ]:
print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)

### <span style="color:#53c653"> 3.4.2. Lower case

La transformation des lettres majuscules contenues dans notre corpus en lettre minuscule participe à *la normalisation et a la préparation des données textuelle*. 

Souvent, c'est une bonne idée : cela permettra par egalement d'harmoniser le mot "Automobile" placé en début de phrase (donc comporte une lettre capitale) et le mot "automobile", placé indifféremment dans la phrase. Mais il y a toujours quelques petites subtilités comme les noms propres ou les acronymes... 

Deux méthodes sont présentées : 
- la méthode naive, qui consiste à tout harmoniser indifféremment de la *"signification"* du mot
- le truecasing qui prend en compte les particularités du langage et laisse en majuscule les noms propres connus par exemple

#### <span style="color:darkgreen">La méthode naive</span>

In [ ]:
start = time.time()
print("Lancement de la transformation 'naive' en lowercase : \n")

print("\n>>> Exemple avant retraitement : ", list(qdf[qdf.index==5629].question))

def transform_to_lowercase(text,var):
    lowercase = []
    for txt in list(text[var]):
        lowercase.append(txt.lower())
    return(pd.Series(lowercase))

LowerCaseNaif = transform_to_lowercase(qdf,"question")


# >>> Selectionnons un exemple en particulier pour pouvoir comparer l'avant et l'après
qdf.question = pd.DataFrame(LowerCaseNaif)        
print(">>> Exemple après retraitement : ", list(qdf[qdf.index==5629].question))

print("\nTemps d'execution : {:.2f} secondes".format(time.time() - start))

In [ ]:
print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)


 #### <span style="color:darkgreen">Le truecasing</span>
 
D'un autre côté, beaucoup de noms propres sont dérivés de noms communs et ne se distinguent donc que par cas, y compris les sociétés (General Motors, Associated Press), les organisations gouvernementales (Fed vs. fed) et les noms de personnes (Bush, Black).

Prenons par exemple cette phrase : "I'm George Bush, I'm American AND I like Cars such as General Motors"
Dans le cas précédent, toutes les lettres capitales seraient remplacées par des lettres minuscules. Or, cette phrase comporte de nombreux noms propres qui pourraient perdre leurs sens suite à cette modification. Typiquement : 
* General Motors -> general motors
Les traitements statistiques qui suivent ces prétraitement ne sauront plus faire la différence entre la grande marque General Motors et des moteurs généraux...

Afin de conserver les lettres capitals lorsque celles ci sont requises, une technique existe : le <span style="color:red">**truecasing**</span>
Cette technique s'appuie sur les résultats issues du pos-tagging...

Un petit exemple avec notre phrase : 

In [ ]:
"""
Quelques exemples de Truecasing
Comme l'execution de ce programme peut etre long pour notre base de donnée, 
on ne le lance que sur une phrase pour comprendre la différence avec la méthode naive
"""

import spacy, re # Import library
nlp = spacy.load('en_core_web_sm')

mysentence = "I'm George Bush, I'm AmeRiCAn. I like CARs such aS General  Motors"
print (mysentence)

doc = nlp(mysentence)
tagged_sent = [(w.text, w.tag_) for w in doc]
nb_NNP = sum(char in ["NNP", "NNPS"] for char in [w.tag_ for w in doc]) 
# On compte le nombre de nom propre présent dans la phrase
normalized_sent = [w.lower() if t not in ["NNP", "NNPS"] else w for (w,t) in tagged_sent]
# Dans cette ligne de commande, on va mettre en minuscule tous les mots qui ne correspondent pas à des noms propres...
normalized_sent[0] = normalized_sent[0].capitalize()
# Cette ligne de commande permet de conserver la lettre capitale du premier mot de la phrase. Cette étape n'est pas essentielle et peut etre supprimée. Elle est juste présentée pour l'exemple.
string = re.sub(" (?=[\.,'!?:;])", "", ' '.join(normalized_sent))
print (string)


### <span style="color:#53c653"> 3.4.3. Abreviation et acronymes

Il n'existe pas de méthode automatique pour gérer ce problème. La technique couramment utilisée consiste à créer un dictionnaire (objet Python) mettant en correspondance la contraction et sa forme entière.

**Remarque** : pour définir des chaines de caractères, l'utilisation de la double quote ( " ) est préférable._

In [ ]:
CONTRACTION_MAP = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" ,
                   "date of birth" : "DOB" , "birth date" : "DOB" ,"birthdate" : "DOB",
                  } 

def Corrections_Abreviations_Acronymes(s):
    # Abreviations
    s=s.replace('Assn.', 'Association')
    s=s.replace('Mt.', 'Mount')
    s=s.replace('Ave.', 'Avenue')
    s=s.replace('St.', 'Street')
    s=s.replace('Dept.', 'Department')
    s=s.replace('No.', 'Number')
    s=s.replace(' no.', ' Number')
    s=s.replace('etc.', ' ')
    s=s.replace(' vs ',' versus ')
    s=s.replace(' vs. ',' versus ')
    s=s.replace('btwn','between')
    s=s.replace('Btwn','Between')
    s=s.replace(' u ',' you ')
    
    #Noms propres, acronymes
    s=re.sub(r'(the USA)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(the U.S.A.)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S.A)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(the US)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(the U.S)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S)(?=[^a-zA-Z])',"America", s)
    s=re.sub(r'(The U.S)(?=[^a-zA-Z])',"America", s)
    s=s.replace('United States of America', 'America')
    s=s.replace('United States', 'America')
    s=s.replace('World Trade Organization', 'WTO')
    s=re.sub(r'( UK)(?=[^a-zA-Z])'," United Kingdom", s)
    s=re.sub(r'( uk)(?=[^a-zA-Z])'," United Kingdom", s)
    s=s.replace('date of birth', 'DOB')
    s=s.replace('birth date','DOB')
    s=s.replace('birthdate','DOB')
    s=s.replace('M.Sc.','MSc')
    s=s.replace('B.Sc.','BSc')
    s=s.replace('B. Tech.','BTech')
    s=s.replace('Master of Science','MSc')
    s=s.replace('Bachelor of Science','BSc')
    s=s.replace('Bachelor of Technology','BTech')
    s=s.replace('Master of science','MSc')
    s=s.replace('Bachelor of science','BSc')
    s=s.replace('Bachelor of technology','BTech')
    s=s.replace('Ph.D.','PhD')
    s=s.replace('Ph.D','PhD')
    s=s.replace('doctorate','PhD')
    s=s.replace('New York','New-York')
    s=s.replace(' NY ',' New-York ')
    s=s.replace('Los Angeles','Los-Angeles')
    s=s.replace(' LA ',' Los-Angeles ')
    s=s.replace('San Francisco','San-Francisco')
    s=s.replace('United Nations','UN')
    s=s.replace('UPSC','civil service')
    s=s.replace('IAS','civil service')
    s=s.replace('upsc','civil service')
    s=s.replace('ias','civil service')
    s=s.replace('Orange is the New Black','OITNB')
    s=s.replace('World War III','WW3')
    s=s.replace('World War 3','WW3')
    s=s.replace('WWIII','WW3')
    s=s.replace('World War II','WW2')
    s=s.replace('World War 2','WW2')
    s=s.replace('WWII','WW2')
    s=s.replace('World War I','WW1')
    s=s.replace('World War 1','WW1')
    s=s.replace('WWI','WW1')
    s=s.replace('Third World War','WW3')
    s=s.replace('third world war','WW3')
    s=s.replace('Third world war','WW3')
    s=s.replace('Second World War','WW2')
    s=s.replace('second world war','WW2')
    s=s.replace('Second world war','WW2')
    s=s.replace('World War I','WW1')
    s=s.replace('World War 1','WW1')
    s=s.replace('WWI','WW1')
    s=s.replace('1st',"first")
    s=s.replace('2nd',"second")
    s=s.replace('3rd',"third")
    s=s.replace('4th',"fourth")
    s=s.replace('5th',"fifth")
    s=s.replace('6th',"sixth")
    s=s.replace('7th',"seventh")
    s=s.replace('8th',"eighth")
    s=s.replace('9th',"nineth")
    #s=s.replace('Wich ', 'Which')
    s=s.replace('Wat ', 'What')
    s=s.replace("(^|\W)\d+($|\W)", " ")
    return s
    

In [ ]:
print("Quelques affichages avant retraitement des contractions : \n --------------------------------------------------------")
print(">>> Exemple avec ' what's ' : \n", list(qdf[qdf.index==21].question))
print(">>> Exemple avec ' date of birth ' : \n", list(qdf[qdf.index==24227].question))
print(">>> Exemple avec ' vs ' : \n", list(qdf[qdf.index==1872].question))
print(">>> Exemple avec ' UPSC ' : \n", list(qdf[qdf.index==38].question))

In [ ]:
def expand_contractions(sentence, contraction_mapping = CONTRACTION_MAP): 
    # mytok = sent_tokenize(sentence)
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),  
                                      flags=re.IGNORECASE|re.DOTALL) 
    def expand_match(contraction): 
        match = contraction.group(0) 
        first_char = match[0] 
        expanded_contraction = contraction_mapping.get(match) if contraction_mapping.get(match) else contraction_mapping.get(match.lower())                        
        try:
            expanded_contraction[1:] 
            return(first_char+expanded_contraction[1:] )
        except TypeError:
            return("")
    
    expanded_sentence = contractions_pattern.sub(expand_match, sentence) 

    return expanded_sentence 

In [ ]:
start = time.time();
print("Lancement de l'expansion des contractions : \n")

res = [expand_contractions(txt, CONTRACTION_MAP) for txt in qdf["question"]]
print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

qdf["question_orig"] = pd.Series(res)

In [ ]:
start = time.time();
print("Corrections_texte")
qdf['question']=qdf['question'].apply(Corrections_Abreviations_Acronymes)
print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

In [ ]:
# >>> Verification qu'il n'y a plus les contractions définies dans notre dictionnaire
print("Quelques affichages après retraitement des contractions : \n --------------------------------------------------------")
print(">>> Exemple avec ' what's ' : \n", list(qdf[qdf.index==21].question_orig))
print(">>> Exemple avec ' date of birth ' : \n", list(qdf[qdf.index==24227].question_orig))
print(">>> Exemple avec ' vs ' : \n", list(qdf[qdf.index==1872].question_orig))
print(">>> Exemple avec ' UPSC ' : \n", list(qdf[qdf.index==38].question_orig))

Quelques controles et affichages : 

In [ ]:
print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)

### <span style="color:#53c653"> 3.4.3. Ponctuation

Cette étape est à faire _après_ avoir fait l'expanding contraction car sinon supprimer les apostrophes ou autres ponctuations qui symbolisent la contraction.

On repère généralement les différentes phrases de la question à l'aide du point suivi d'un espace.  Le problème est que cette convention n'est pas toujours respectée !!! Typiquement, lorsqu'il s'agit de phrase interrogative (la ponctuation finale est le point d'interrogation "?") ou exclamatives (cette fois, c'est le point d'exclamation "!"). 

Comment faire dans ces conditions ? 
une méthode simple consiste à décoller le point du mot qui suit. "Simple!" me direz-vous? Trop à vrai dire... Et c'est oublier qu'il existe toujours des exceptions ! En effet, il y a des points collés *légitimes*, comme le point dans une URL ou dans une extension *fichier*.pdf. 

La liste des ponctuations est la suivante : 

In [ ]:
import string
ponctuation = set(string.punctuation)
print(ponctuation)

In [ ]:
def Corrections_Ponctuations(s):
    s=s.replace('/',' ')
    s=s.replace('"','')
    s=s.replace("'",' ')
    s=s.replace("'",' ')
    s=s.replace("``",'')
    s=s.replace("''",'')
    s=s.replace('~',' ')
    s=s.replace('...','. ')
    s=s.replace('..','. ')
    return s

#On prepare les extensions

f2={'io', 'in', 'is', 'js', 'it', 'us', 'fm', 'tv', 'ca', 'vn', 'no', 'fr', 'sc', \
    'la', 'de', 'ed', 'ex', 'eu', 'e', 'ru', 'be', 'py', 'ai', 'as', 'im', 'nl', 'gv', \
    'gq', 'ga', 'ts', 'tk', 'dt', 'em', 'cs', 'cr', 'jp', 'cn', 'cm', 'cd', 'pt', 'pz', \
    'la', 'pg', 'pl', 'hr', 'uk', 'aa', 'va', 'an', 'ar', 'at', 'ie', 'nd', 'sh', 'ke', \
    'st', 'se', 'sd'}


f3={'com', 'net', 'org', 'exe', 'dll', 'app', 'php', 'pdf', 'edu', 'emz', 'eng', 'zip',\
    'xml', 'png', 'jpg', 'inc', 'rtf', 'cda', 'dbb', 'mp3', 'wrf', 'm4v', 'cue', 'mov',\
    'psd', 'dmg', 'cpp', 'bak', 'vhd', 'dir', 'htm', 'odt', 'lnk', 'tmp', 'dds', 'rpm',\
    'ddf', 'obj', 'css', 'vtf', 'jar', 'api', 'ppt', 'mbp', 'swf', 'gif', 'wav', 'jmx',\
    'tec', 'qsv', 'nh3', 'flp', 'dta', 'tar', 'rar', 'avi', 'hw6', 'mid', 'csv', 'doc'}


f4={'html', 'java', 'jpeg', 'json', 'conf', 'arch', 'fxml', 'yaml', 'vmdk', 'aspx', \
    'addr', 'proc', 'ajax', 'flac', 'adhd', 'nrkt'}

def Decollage_De_Points(s):
    point = s.find('.') 
    #if point!=-1:
    #    print(s)
    while (point>1) and (point!=len(s)-1) : 
        if (s[point+1]!=' ') and (s[point+1]!='.') :
            debut_mot = s.rfind(' ',0,point) + 1
            fin_mot = s.find(' ',point+1) - 1
     #       print(debut_mot, fin_mot)
            if fin_mot==-2:
                fin_mot = len(s)-1
            while s[fin_mot] in ponctuation:
                fin_mot -= 1
            mot = s[debut_mot:fin_mot+1]
     #       print(mot)
            legitime = False
            if (point+2==fin_mot) and (mot[-2:] in f2):
                legitime = True
            if (point+3==fin_mot) and (mot[-3:] in f3):
                legitime = True
            if (point+4==fin_mot) and (mot[-4:] in f4):
                legitime = True
            if ('http' in mot) or ('www' in mot):
                legitime = True
            if not legitime:
                s=s[:point+1]+' '+s[point+1:]
        if point==len(s)-1:
            point = -1
        else:
            point = s.find('.',point+1)
    return s

def remove_before_token(sentence, keep_apostrophe = False):
    
    sentence = sentence.strip()
    
    if keep_apostrophe:
        PATTERN = re.compile("(\.|\!|\?|\(|\)|\-|\$|\&|\*|\%|\@|\~)")
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    else :
        PATTERN = r'[^a-zA-Z0-9]'
        filtered_sentence = re.sub(PATTERN, r' ', sentence)
    return(filtered_sentence)


In [ ]:
print("Lancement des procedures relatives aux ponctuations : \n")

start = time.time();

print("Quelques affichages avant retraitement des contractions : \n --------------------------------------------------------\n", 
      list(qdf[qdf.index==1].question))

print("\n   >>> Correction de la ponctuation...")
qdf['question_orig']=qdf['question'].apply(Corrections_Ponctuations)

print("   >>> Décollage de points...")
qdf['question_orig']=qdf['question'].apply(Decollage_De_Points)

print("   >>> Suppression de la ponctuation inutile...\n")
text = [remove_before_token(txt, keep_apostrophe = False) for txt in list(qdf.question)]
qdf["question"] = pd.DataFrame(text)

print("Quelques affichages après retraitement des contractions : \n --------------------------------------------------------\n", 
      list(qdf[qdf.index==1].question))
            
print("Temps d'execution : {:.2f} secondes".format(time.time() - start))


Quelques controles et affichages : 

In [ ]:
print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)

### <span style="color:#53c653"> 3.4.4. Decoupage des phrases multiples

Certaines "questions" peuvent contenir en réalité plusieurs phrases. 
Par exemple : 



In [ ]:
print("Lancement des procedures relatives aux découpage de phrases multiples : \n")

start = time.time();

detecteur_de_phrases = nltk.data.load('tokenizers/punkt/english.pickle')
qdf['questions_multiples']=qdf['question'].apply(detecteur_de_phrases.tokenize)
qdf['n_questions']=qdf['questions_multiples'].apply(len)
qdf.head()

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

In [ ]:
print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)

In [ ]:
print(qdf['question_orig'][qdf['n_questions']==0])

##Que faire ??
qdf.head()
list(qdf[qdf.index==55096].question)

""" TO DO - a revoir """

Quelques petits soucis encore ...

In [ ]:
#plt.hist(qdf['n_questions'],bins=22)
#plt.show()
print(qdf['n_questions'].value_counts())
print(qdf['question'][qdf['n_questions']==8])


# <span style="color:#006600">4. Extraction des mots et correcteurs orthographique 

Les nombreuses études déjà réalisées, tant en NLP qu’en linguistique pure, s’accordent toutes plus ou moins sur un schéma relativement similaire : 

```
1. Les tokens et la tokenisation
2. Le correcteur orthographique 
3. Les stopwords
4. Les pos-tagging
5. Lemmatisation
```

##  <span style="color:#39ac39"> 4.1. Les tokens et la tokenisation
Cette première étape consiste a découper le texte en plusieurs **<span style="background-color: #ccffcc;">tokens</span>**.  Les tokens sont les éléments porteurs de sens les plus simples au sein d'une phrase (autrement dit, les mots!). 

Etant donné que les espaces entre les mots permettent de les délimiter entre eux, une manière de *tokenizer* est donc de séparer les mots par rapport à ces espaces. 

** Un petit exemple ** : 

```Le chat dort.```

Cette phrase comporte trois mots séparés par des espaces, on peut donc en déduire qu'il y a trois tokens. 

Si on considère maintenant : *Aujourd'hui, le chat dort*. Combien y a-t-il de tokens ? et combien y a-t-il de tokens dans y *Combien y a-t-il de tokens a-t-il* ?
Dans le cas de *a-t-il*, *va-t’en* etc, on comprends qu'il y a plusieurs tokens et pourtant... aucun espace ! On aurait envie dans ce cas d'étendre notre règle précédente et de considérer que certains signes de ponctuation peuvent etre utilisé comme séparateur de mot... 
Mais cette règle conduirait à considérer le mot *Aujourd'hui* comme deux tokens, ... alors qu'il s'agit bien d'un seul et même token. 


Un petit exemple pratique : 

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

EXAMPLE_TEXT = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."

print(sent_tokenize(EXAMPLE_TEXT))


**Quelques commentaires **: 
1. La ponctuation est considérer comme un token à part entière
2. La séparation de *shouldn't* donne les tokens *should* et *n't*. Cela signifie que la procédure utilisée comprend bien ces deux mots concaténés ne sont pas porteur d'un sens unique (avec respectivement un verbe et une négation).
3. Le mot *pinkish-blue* est traité comme un seul et unique token


##  <span style="color:#39ac39"> 4.2. Le correcteur orthographique

** _Remarque_** : On ne doit pas confondre le ** <span style="background-color:#ccffcc"> correcteur orthographique </span>** et le ** <span style="background-color:#ccffcc"> correcteur grammatical </span>**. Le correcteur orthographique compare les mots du texte aux mots d'un dictionnaire. Si les mots du texte sont dans les dictionnaires, ils sont acceptés, sinon une ou plusieurs propositions de mots proches sont faites par le correcteur orthographique. Le correcteur grammatical vérifie que les mots du texte, bien qu'ils soient dans les dictionnaires, sont conformes aux règles de grammaire (accords, ordre des mots, etc.) et aux règles de la sémantique (phrase ayant un sens, absence de confusion d'homophones, etc.).

Deux méthodes présentées : 
1. Méthode basée sur un calcul de distance entre le mot et les suggestions (distance de Levenshtein)
2. Méthode basée sur la probabilité qu'une des suggestions soient la bonne (algorithme de Norvig)

### <span style="color:#53c653"> Methode 1 : En utilisant la distance de Levenshtein

Plusieurs distance dans le module nltk : avec edit_distance qui correspond a la distance de Levenshtein (source : http://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/ ) 
* **Hamming** : présuppose que deux chaines de caractères sont des mêmes longueurs
* **Levenshtein** : contrairement la distance de Hamming, la distance de Levenshtein permet de comparer des chaines dont la longueur est différente. La distance est calculée en déterminant le nombre de transformation qu'il est nécessaire pour passer d'une chaine de caractère A a la chaine de caractère B. Ces transformations peuvent etre : 
    * Substitution 
    * Insertion 
    * Suppression
    Par exemple : La distance de Levenshtein entre "rain" et "shine" est de 3 puisqu'il y a deux substitutions et une insertion :
    "rain" -> "sain" -> "shin" -> "shine". 
* **Damerau-Levenshtein**
* **Jaro Winkler**


In [ ]:
def hamming_distance(s1, s2):
    s1 = re.sub('[^A-Za-z0-9]+', '', s1); print(len(s1))
    s2 = re.sub('[^A-Za-z0-9]+', '', s2); print(len(s2))
    assert len(s1) == len(s2)
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))

def damerau_levenshtein(s1, s2):
    d = {}
    len_s1 = len(s1)
    len_s2 = len(s2)
    for i in range(-1, len_s1 + 1):
        d[(i, -1)] = i + 1
    for j in range(-1, len_s2 + 1):
        d[(-1, j)] = j + 1

    for i in range(len_s1):
        for j in range(len_s2):
            if s1[i] == s2[j]:
                cost = 0
            else:
                cost = 1
            d[(i, j)] = min(
                d[i - 1, j] + 1,  # Deletion
                d[i, j - 1] + 1,  # Insertion
                d[i - 1, j - 1] + cost,  # Substitution
            )

            if i and j and s1[i] == s2[j - 1] and s1[i - 1] == s2[j]:
                d[i, j] = min(d[i, j], d[i - 2, j - 2] + cost)  # transposition
    return d[len_s1 - 1, len_s2 - 1]

In [ ]:
import hunspell
from nltk.metrics import *
import numpy as np
from scipy.spatial.distance import pdist
from Levenshtein import jaro_winkler

hobj = hunspell.HunSpell("/usr/share/hunspell/en_US.dic", "/usr/share/hunspell/en_US.aff")

def correct_word(myword):
    res = [x for x in list(hobj.suggest(myword)) if edit_distance(myword,x) == 1 ] ; print(res)
    if len(res) == 1:
        correct = res
    if len(res) == 0: 
        res = [x for x in list(hobj.suggest(myword)) if edit_distance(myword,x) == 2 ] 
        if len(res) == 1: correct = res
        elif len(res) == 2: 
            tmp = [jaro_winkler(myword,x) for x in res]; 
            correct = res[tmp.index(max(tmp))] # mettre le max au sens de jaro winkler
        else : correct = None
    if len(res) > 1:
            tmp = [jaro_winkler(myword,x) for x in res]; 
            correct = res[tmp.index(max(tmp))] # mettre le max au sens de jaro winkler
    print("The best candidate for {} is : {}".format(myword,correct))
    return(correct)

In [ ]:
# Quelques exemples : 
correct_word("mywords")
correct_word("spooky")
correct_word("mustkaes")

Avec cette méthode, le dernier mot n'est pas reconnu 

### <span style="color:#53c653"> Methode 2 : Peter Norvig
Cette méthode tente de choisir  la correction d'orthographe la plus probable pour un mot donné. Il n'y a aucun moyen de savoir avec certitude (par exemple, faut-il corriger les *"lates" *en *"late"* ou *"latest"* ou *"lattes"* ou ...?), Ce qui suggère que nous utilisons des probabilités. Nous essayons de trouver la correction, parmi toutes les corrections possibles, qui maximise la probabilité que ce soit la correction voulue, étant donné le mot original.

Cette probabilité est déterminée en comptant le nombre de fois que chaque mot apparaît dans un fichier texte d'environ un million de mots, appelé **_big.txt_**. Il s'agit d'une concaténation : 
* d'extraits de livres du domaine public du Projet Gutenberg et 
* de listes de mots les plus fréquents de Wiktionary et du British National Corpus

Pour plus d'information : https://norvig.com/spell-correct.html 

In [ ]:
import re
import nltk
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('../input/spelling/big.txt').read()))

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))
    

In [ ]:
# Exemple avec des mots au hasard 
print(correction('mywords'))
print(correction('spooky'))
print(correction("mustkaes"))

##  <span style="color:#39ac39"> 4.3. Les stopwords

Les ** <span style="background-color:#ccffcc"> " stopwords "</span>** sont en très grande partie composée de mots qui n’ont pas de sess en eux-mêmes mais qui sont utilisés dans la constructions des phrases (ex. : prépositions, pronoms, verbes uxilaires, articles). 

Une des difficultés est qu'il n'existe pas de liste universelle de stopwords. Ils sont **caractéristiques d’une langue**. En effet, il est assez peu probable que vous utilisiez dans une phrase francaise *the* ou de *are* , sauf si vous buvez du thé tout en discutant de métrologie (l'are est une unité de mesure de superficie).

La librairie <span style="background-color:#ccffcc"> NLTK </span> fournit une liste des stopwords les plus communs. Pour accéder a cette liste prédéfinie : 

```
from nltk.corpus import stopwords
stopwords.words('english')
```

**Quelques exemples** : 'i','me','my','myself','we','our','ours','ourselves','you',"you're","you've","you'll",'because','as','until','while','of','at','by','for','with','about','against',

Et en francais ? Ca existe aussi ! 
```
from nltk.corpus import stopwords
stopwords.words('french')
```
**Quelques exemples** :  'au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'je', 'la', 'le', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', etc...

Vous pouvez également ajouter votre stopwords, voire même supprimer quelques uns qui sont présents dans la liste de NLTK, en utilisant respectivement les lignes de commandes suivantes : 

```
stop += ['would']
stop.remove('how')
```

Comme dans notre cas, les mots interrogatifs nous interesses (puisqu'il s'agit de faire la correspondance entre des questions!), nous allons conserver les "What", "Who" etc...

In [ ]:
#  Apportons notre touche personnelle sur les stopwords...
stop = stopwords.words('english')
stop.remove('what')
stop.remove('which')
stop.remove('who')
stop.remove('whom')
stop.remove('when')
stop.remove('where')
stop.remove('why')
stop.remove('how')
# A faire autrement == traitement de la ponctuation et non pas stopwords
# stop += [',','.','...','(',')','[',']','{','}','!','?',';',':','*','=','<','>','&','$','+',"&",'-']
stop += ['would']
stopword_question = set(stop)

# Affichages : 
print("Les stopwords conservés sont les suivants : \n ", stopword_question)

##  <span style="color:#39ac39"> 4.4. Le pos-tagging (Part of Speech tagging)

Le <span style="background-color:#ccffcc"> **  pos-tagging ** </span> consiste à étiqueter les mots comme noms, adjectifs, verbes ... etc. Encore plus impressionnant, il *tag* également si le verbe est conjugé, s'il y a des accords... et plus encore.  L'étiquetage est dépendant de la qualité du texte (pas de faute d'orthographe par exemple !!)

**<span style="color:darkgreen">Petits exemple _sans_ faute </span>** : 
![POSTAG_CORRECT](https://github.com/sdaymier/NLP/blob/master/POSTAG_Correct.PNG?raw=true "Logo Title Text 1")

**<span style="color:darkgreen">Petits exemple _avec_ des fautes</span>** : 
![POSTAG_INCORRECT](https://github.com/sdaymier/NLP/blob/master/POSTAG_Incorrect.PNG?raw=true "Logo Title Text 1")


Voici une liste des tags, ce qu'ils signifient, et quelques exemples (liste non exhaustive) :
 

| Tag | Meaning | English Examples |
| ------------- |:----------------:|:----------------:|
| ADJ | adjective | new, good, high, special, big, local |
| ADP | adposition  | on, of, at, with, by, into, under |
| ADV  | adverb | really, already, still, early, now |
| CONJ  | conjunction |  and, or, but, if, while, although  |
|  DET | determiner, article  | the, a, some, most, every, no, which  |
| NOUN | noun | noun year, home, costs, time, Africa   |
| NUM |numeral | twenty-four, fourth, 1991, 14:24  |
| PRT  |particle  |  at, on, out, over per, that, up, with   |
| PRON | pronoun |   he, their, her, its, my, I, us  |
| VERB | verb |  is, say, told, given, playing, would  |
| "." | punctuation | " . , ; !"  |
|X | other  | ersatz, esprit, dunno, gr8, univeristy  |

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

EXAMPLE_TEXT = "Hello Mr. Smith, how are you doing today? The weather is great, and Python is awesome. The sky is pinkish-blue. You shouldn't eat cardboard."
words = nltk.word_tokenize(EXAMPLE_TEXT)
tagged = nltk.pos_tag(words)

print(tagged)

##  <span style="color:#39ac39"> 4.5. Lemmatisation

La  <span style="background-color:#ccffcc">lemmatisation</span> consiste a analyser les termes de manière a identifier sa forme canonique (lemme) qui existe réellement. L’idée est de réduire les différentes formes (pluriel, féminin, conjugaison…) en une seule. La technique fait à la fois référence a un dictionnaire et à l’analyse morphosyntaxique des mots. Elle est spécifique à chaque langue.

**Un petit exemple ** : Le mot «** invisible** » est composé de trois morphèmes : **_in- vis- ible_**, « voir » étant le lemme du mot. A partir de ces éléments, on peut déterminer comment le lemme a été altéré et pourquoi. Dans notre cas : 
* le préfixe «* in *» apporte une connotation négative au verbe voir 
* le suffixe « *ible* » exprime la capacité à faire quelque chose.

In [ ]:
def Traducteur_Tag_Pour_Lemmatizer(tag):
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

def lemmatise(phrase_taggee):
    return [nltk.stem.WordNetLemmatizer().lemmatize((x[0].lower()), Traducteur_Tag_Pour_Lemmatizer(x[1])) \
            for x in phrase_taggee if x[0].lower() not in stopword_question]


##  <span style="color:#39ac39"> 4.6. Application de l'extracteur de mot (driver des fonctions précédentes)

La fonction définie ci dessous est le driver des étapes à suivre , que nous avons vu en détail précédemment.: 


** WARNING ** : Très long !!!
Exemple : Temps d'execution : 62.61 secondes sur la base de train avec uniquement les tokens

In [ ]:
# Version complète
def extractWords(phrases_multiples):
    q_multiples_tokenizees  = [nltk.word_tokenize(x) for x in phrases_multiples] # 1. Tokenization
    q_multiple_correct      = [correction(x) for x in phrases_multiples]         # 2. Correcteur orthographique
    q_multiples_taggees     = [nltk.pos_tag(x) for x in q_multiples_tokenizees]  # 3. Stopwords et pos tagging
    q_multiples_lemmatisees = [lemmatise(x) for x in q_multiples_taggees]        # 4. Lemmatization

    return [x for x in q_multiples_lemmatisees if len(x)>0]

In [ ]:
# Version allégée
def extractWords(phrases_multiples):
    q_multiples_tokenizees  = [nltk.word_tokenize(x) for x in phrases_multiples] # 1. Tokenization
    # q_multiple_correct      = [correction(x) for x in phrases_multiples]         # 2. Correcteur orthographique
    # q_multiples_taggees     = [nltk.pos_tag(x) for x in q_multiples_tokenizees]  # 3. Stopwords et pos tagging
    # q_multiples_lemmatisees = [lemmatise(x) for x in q_multiples_taggees]        # 4. Lemmatization

    return [x for x in q_multiples_tokenizees if len(x)>0]

In [ ]:
start = time.time();
print("Lancement des procedures relatives aux extractions de mots et correcteurs orthographique : \n")

qdf["sac_de_mots"]=qdf["questions_multiples"].apply(extractWords)

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

Quelques controles et affichages : 

In [ ]:
print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)

In [ ]:
# qdf.to_csv("QDF_TRAIN_PostTreatment.csv", index=False)

Quelques affichages : 

In [ ]:
qdf[100:110]

# <span style="color:#006600"> 5. Autres features : 
##  <span style="color:#39ac39"> 5.1. Repérer le mot interrogatif

In [ ]:
mots_interrogatifs=['what','which','how','when','where','who','why','whom']

start = time.time();
print("Lancement des procedures relatives aux extractions des mots interrogatifs : \n")


def Trouve_MI(SacDeMots):
        
    for mot in SacDeMots:
        if mot in mots_interrogatifs:
            return mot
    
    return 'que_dalle'

def Liste_MI(ListePhrases):
    return [Trouve_MI(x) for x in ListePhrases]
    
    
qdf['mots_interrogatifs']=qdf['sac_de_mots'].apply(Liste_MI)

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

print("Nombre de variable : ", len(list(qdf.columns)))
print("Liste des variables : ", list(qdf.columns))
qdf.head(5)

##  <span style="color:#39ac39"> 5.2. Creation de feature avec Word2Vec

In [ ]:
import gensim
../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin
model = gensim.models.KeyedVectors.load_word2vec_format('../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)
indecision = model.similarity('chess','sausage')

def similarite_mot(mot1, mot2):
    if mot1==mot2:
        return 1.0
    if (mot1 in model.vocab) and (mot2 in model.vocab):
        return model.similarity(mot1, mot2)#vec_mot1.dot(model[mot2])
    return indecision

def similarite_phrase(p1,p2):
    similarite=1
    n_mots=0

    p_courte=p1
    p_longue=p2
    if len(p2)<len(p1):
        p_courte=p2
        p_longue=p1
    if len(p_courte)==0.1:
        return indecision   ### 0 ?
    
    #print("p1 :",p_courte)
    #print("p2 :",p_longue)
    for mot in p_courte:
        #print(mot)
        n_mots+=1
        ressemblances = [similarite_mot(mot, mot2) for mot2 in p_longue]
        #print(ressemblances)
        similarite *= max(ressemblances)        
        
    return similarite**(1.0/n_mots)    


def similarite_Word2Vec(ligne):
    sdm1 = qdf['sac_de_mots'][qdf['id']==ligne['qid1']].iloc[0]
    sdm2 = qdf['sac_de_mots'][qdf['id']==ligne['qid2']].iloc[0]
    #print(ligne['qid1'],sdm1)
    #print(ligne['qid2'],sdm2)
    
    if len(sdm1)==0:
        return indecision #### ?
    if len(sdm2)==0:
        return indecision #### ?
    
    similarites = []
    
    for p1 in sdm1:
        for p2 in sdm2:
            similarites += [similarite_phrase(p1,p2)]
    #print(similarites)
    return max(similarites)

def similarite_phrase_2(p1,p2):
    similarite=1
    n_mots=0
    
    
    p_courte=p1
    p_longue=p2
    if len(p2)<len(p1):
        p_courte=p2
        p_longue=p1
    if len(p_courte)==0.1:
        return indecision   ### 0 ?
    
    #print("p1 :",p_courte)
    #print("p2 :",p_longue)
    for mot in p_longue:
        #print(mot)
        n_mots+=1
        ressemblances = [similarite_mot(mot, mot2) for mot2 in p_courte]
        #print(ressemblances)
        similarite *= max(ressemblances)        
        
    return similarite**(1.0/n_mots)    


def similarite_Word2Vec_2(ligne):
    sdm1 = qdf['sac_de_mots'][qdf['id']==ligne['qid1']].iloc[0]
    sdm2 = qdf['sac_de_mots'][qdf['id']==ligne['qid2']].iloc[0]
    #print(ligne['qid1'],sdm1)
    #print(ligne['qid2'],sdm2)
    
    if len(sdm1)==0:
        return indecision #### ?
    if len(sdm2)==0:
        return indecision #### ?
    
    similarites = []
    
    for p1 in sdm1:
        for p2 in sdm2:
            similarites += [similarite_phrase_2(p1,p2)]
    #print(similarites)
    return max(similarites)

In [ ]:
"""
# Lancement des fonctions
df_train['similarite_w2vec']=df_train.progress_apply(similarite_Word2Vec, axis=1)
df_train['similarite_w2vec_2']=df_train.progress_apply(similarite_Word2Vec_2, axis=1)
"""

In [ ]:
df_Word2vec = pd.read_csv('../input/featuresword2vec/features_w2vec.csv')
print("Dimension des features de Word2Vec : ", df_Word2vec.shape)
print("Dimension des features de df_train : ", df_train.shape)

# Concatenation des bases : 
df_train["id"] = df_train.index
df_train = df_train.merge(df_Word2vec, left_on='id', right_on='Unnamed: 0', how='outer')
df_train.drop(["id",'Unnamed: 0'], axis = 1, inplace= True)

print("Dimension des features de df_train : ", df_train.shape)

In [ ]:
plt.hist(df_train[(df_train['is_duplicate']==1) & ~(df_train['similarite_w2vec'].isnull())]['similarite_w2vec'])
plt.show()
plt.hist(df_train[(df_train['is_duplicate']==0) & ~(df_train['similarite_w2vec'].isnull())]['similarite_w2vec'])
plt.show()
plt.hist(df_train[(df_train['is_duplicate']==1) & ~(df_train['similarite_w2vec_2'].isnull())]['similarite_w2vec_2'])
plt.show()
plt.hist(df_train[(df_train['is_duplicate']==0) & ~(df_train['similarite_w2vec_2'].isnull())]['similarite_w2vec_2'])
plt.show()
df_train['similarite']=df_train['similarite_w2vec']*df_train['similarite_w2vec_2']
df_train['similarite'].loc[df_train['similarite'].isnull()]=indecision*indecision
plt.hist(df_train[(df_train['is_duplicate']==1) ]['similarite'])
plt.show()
plt.hist(df_train[(df_train['is_duplicate']==0) ]['similarite'])
plt.show()


Extract pour la formation : 

In [ ]:
"""
noformule = qdf.id[qdf['formule'].apply(len)==0][0:10].tolist()
formule = qdf.id[qdf['formule'].apply(len)>0][0:10].tolist() 
nonumber = qdf.id[qdf['nombre'].apply(len)==0][0:10].tolist()
number = qdf.id[qdf['nombre'].apply(len)>0][0:5].tolist()
noquestion = qdf.id[qdf['n_questions']==0][0:10].tolist()
question1 = qdf.id[qdf['n_questions']==1][0:10].tolist()

list_id = noformule + formule + nonumber + number + noquestion + question1
set_id = set(list_id)

df_train.iloc()
df1 = df_train.loc[df_train['qid1'].isin(set_id)]
df2 = df_train.loc[df_train['qid2'].isin(set_id)]

df = pd.concat([df1,df2])
print(df.shape)

df.to_csv("Extract_before_formation.csv", index=False)
qdf.to_csv("QDF_TRAIN_PostTFIDF.csv", index=False)
"""

# <span style="color:#006600"> 6. Modelisation

##  <span style="color:#39ac39"> 6.1 Reconstitution de la base d'apprentissage

In [ ]:
print("Récupération des id")
df_qid1 = qdf.loc[qdf['id'].isin(df_train.qid1.tolist())]
df_qid2 = qdf.loc[qdf['id'].isin(df_train.qid2.tolist())]
print("Shape de df_qid1", df_qid1.shape)
print("Shape de df_qid2", df_qid2.shape)

df_qid1.drop(["index"], axis = 1, inplace = True)
df_qid2.drop(["index"], axis = 1, inplace = True)

In [ ]:
print("Renommer les colonnes : ")
df_qid1.columns = ["id_1","question_orig_1","question_1","formule_1","nombre_1","questions_multiples_1",
                   'n_questions_1', 'mots_interrogatifs_1', 'sac_de_mots_1' 
                   # ,'tous_les_mots_1', 'tous_les_mots_uniques_1', 
                   # 'tf_idf1_1','tf_idf2_1', 'tf_idf3_1'
                  ]
  
df_qid2.columns = ["id_2","question_orig_2","question_2","formule_2","nombre_2","questions_multiples_2",
                   'n_questions_2', 'mots_interrogatifs_2', 'sac_de_mots_2',  
                   # 'tous_les_mots_2', 'tous_les_mots_uniques_2', 
                   # 'tf_idf1_2','tf_idf2_2', 'tf_idf3_2'
                  ]
print("Affichage des noms de colonnes : \n", list(df_qid1.columns))

In [ ]:
# Merge df 1
df_qid1.id_1 = df_qid1.id_1.astype("int64") ; 
df_qid2.id_2 = df_qid2.id_2.astype("int64") ;
# Suppression des colonnes inutiles 
df_qid1.drop(['question_orig_1'], axis=1, inplace=True)
df_qid2.drop(['question_orig_2'], axis=1, inplace=True)
# Affichage des caractéristiques des tables
print("Info sur df_qid1 : \n", df_qid1.info())
print("Info sur df_qid2 : \n", df_qid2.info())

In [ ]:
print("Constitution de la base de modélisation :")
df_model = df_train; print(df_model.shape)

print("\n   >>> Ajout des données de qdf1 : ")
df_model = df_model.merge(df_qid1, left_on = "qid1", right_on = "id_1") ; print(df_model.shape)
del df_qid1; gc.collect()

print("\n   >>> Ajout des données de qdf2 : ")
df_model = df_model.merge(df_qid2, left_on = "qid2", right_on = "id_2") ; print(df_model.shape)
del df_qid2; gc.collect()

print("\n   >>> Suppression des variables inutiles: ")
df_model.drop(["id_1","id_2"], axis = 1, inplace = True)

df_model.sort_values(['qid2'], ascending=[True], inplace=True)
print(df_model.columns)
print("End")

In [ ]:
print("Suppression des variables inutiles : ")
list_column_drop = ['qid1','qid2','question1','question2',
                    "questions_multiples_1","questions_multiples_2", 
                    "sac_de_mots_1",
                    # ,"tous_les_mots_1","tous_les_mots_uniques_1",
                    # "tf_idf1_1","tf_idf2_1","tf_idf3_1",
                    "sac_de_mots_2",
                    # ,"tous_les_mots_2","tous_les_mots_uniques_2",
                    # "tf_idf1_2","tf_idf2_2","tf_idf3_2",
                    "mots_interrogatifs_1","mots_interrogatifs_2",
                    "formule_1","nombre_1","formule_2","nombre_2"]
df_model.drop(list_column_drop, axis = 1, inplace = True)

# Visualistion du résultat
print(df_model.columns.tolist())
df_model.head(5)

In [ ]:
def get_weight(count, eps=10000, min_count=2):
    return 0 if count < min_count else 1 / (count + eps)

def word_shares(row):
    q1 = set(str(row['question_1']).lower().split())
    q1words = q1.difference(stops)
    if len(q1words) == 0:
        return '0:0:0:0:0'

    q2 = set(str(row['question_2']).lower().split())
    q2words = q2.difference(stops)
    if len(q2words) == 0:
        return '0:0:0:0:0'

    q1stops = q1.intersection(stops)
    q2stops = q2.intersection(stops)

    shared_words = q1words.intersection(q2words)
    shared_weights = [weights.get(w, 0) for w in shared_words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]

    R1 = np.sum(shared_weights) / np.sum(total_weights) #tfidf share
    R2 = len(shared_words) / (len(q1words) + len(q2words)) #count share
    R31 = len(q1stops) / len(q1words) #stops in q1
    R32 = len(q2stops) / len(q2words) #stops in q2
    return '{}:{}:{}:{}:{}'.format(R1, R2, len(shared_words), R31, R32)

##  <span style="color:#39ac39"> 6.2. Variables additionnelles

In [ ]:
start = time.time();
print("Creation de nouvelles features : \n")

train_qs = pd.Series(df_model['question_1'].tolist() + df_model['question_2'].tolist()).astype(str)
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}
stops = set(stopwords.words("english"))

print("   >>> Procedure")
df_model['word_shares'] = df_model.apply(word_shares, axis=1, raw=True)


print("   >>> Retraitement de la base : ")
df_model['word_match']       = df_model['word_shares'].apply(lambda x: float(x.split(':')[0]))
df_model['tfidf_word_match'] = df_model['word_shares'].apply(lambda x: float(x.split(':')[1]))
df_model['shared_count']     = df_model['word_shares'].apply(lambda x: float(x.split(':')[2]))

df_model['stops1_ratio']     = df_model['word_shares'].apply(lambda x: float(x.split(':')[3]))
df_model['stops2_ratio']     = df_model['word_shares'].apply(lambda x: float(x.split(':')[4]))
df_model['diff_stops_r']     = df_model['stops1_ratio'] - df_model['stops2_ratio']

df_model['len_q1'] = df_model['question_1'].apply(lambda x: len(str(x)))
df_model['len_q2'] = df_model['question_2'].apply(lambda x: len(str(x)))
df_model['diff_len'] = df_model['len_q1'] - df_model['len_q2']

df_model['len_char_q1'] = df_model['question_1'].apply(lambda x: len(str(x).replace(' ', '')))
df_model['len_char_q2'] = df_model['question_2'].apply(lambda x: len(str(x).replace(' ', '')))
df_model['diff_len_char'] = df_model['len_char_q1'] - df_model['len_char_q2']

df_model['len_word_q1'] = df_model['question_1'].apply(lambda x: len(str(x).split()))
df_model['len_word_q2'] = df_model['question_2'].apply(lambda x: len(str(x).split()))
df_model['diff_len_word'] = df_model['len_word_q1'] - df_model['len_word_q2']

df_model['avg_world_len1'] = df_model['len_char_q1'] / df_model['len_word_q1']
df_model['avg_world_len2'] = df_model['len_char_q2'] / df_model['len_word_q2']
df_model['diff_avg_word'] = df_model['avg_world_len1'] - df_model['avg_world_len2']

df_model.drop(["word_shares"], axis = 1, inplace = True)

"""
df_model['common_words'] = df_model.apply(lambda x: 
                                    len(set(str(df_model['question_1']).lower().split()).intersection(set(str(df_model['question_2']).lower().split()))),
                                    axis=1)

df_model['exactly_same'] = (df_model['question_1'] == df_model['question_2']).astype(int)
df_model['duplicated'] = df_model.duplicated(['question_1','question_2']).astype(int)
"""

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

##  <span style="color:#39ac39"> 6.3. TF IDF sur les phrases nettoyées

### <span style="color:#53c653"> 6.3.1. Quelques petits rappel de la dernière fois


Pour certaines tâches spécifiques impliquant plusieurs textes, (comparaison de textes, ou recherche par-mot clé), il vaudra mieux utiliser le modèle de probabilité TF-IDF qui est plus performant que le modèle Unigram. 

![TFIDF](https://github.com/sdaymier/NLP/blob/master/TFIDF.PNG?raw=true "Logo Title Text 1")

Dans ce modèle, un mot est important s'il apparaît 
* Beaucoup dans le corpus,
* Mais dans peu de textes.
Cela signifie qu'il est vraiment porteur d'un sens fort, qui permet de discriminer entre les textes. 

### <span style="color:#53c653"> 6.3.2. Methode 1 : "A la mano"
Comment allons nous calculer ?
* **TF** : Indique la fréquence du mot dans chaque document du corpus. C'est le rapport entre le nombre de fois que le mot apparaît dans un document et le nombre total de mots dans ce document. Il augmente à mesure que le nombre d'occurrences de ce mot dans le document augmente. Chaque document a son propre tf.
* **IDF** : utilisé pour calculer le poids des mots rares dans tous les documents du corpus. Les mots qui apparaissent rarement dans le corpus ont un score IDF élevé.
 
 
Avec notre base de donnés : chaque phrase est un _document_. 

##### Creation du vocabulaire

In [ ]:
#Creation du vocabulaire
from functools import reduce

def Fusionne_Liste_De_Liste_De_Mots(llm):
    return [x for lm in llm for x in lm] 

def Unique(l):
    return list(set(l))

start = time.time();
print("Construction du vocabulaire : \n")

print("   >>> Recupération de tous les mots présents dans le sac de mot, créé sur toutes les phrases du corpus : ")
qdf["tous_les_mots"] =  qdf['sac_de_mots'].apply(Fusionne_Liste_De_Liste_De_Mots)
print("   >>> Creation d'une liste de tous les mots unique : ")
qdf["tous_les_mots_uniques"] =  qdf['tous_les_mots'].apply(Unique)
print("   >>> Affichage du résultat : ")
print(qdf['tous_les_mots'].head(5))

# Création d'un objet Python permettant de réaliser des boucles efficientes (ce sont des itérables)
#    >>> Pour les mots (avec doublons)
#    >>> Pour la liste de mot unique
vocab = itertools.chain.from_iterable(qdf['tous_les_mots'].values)
vocab_pour_doc_freq = itertools.chain.from_iterable(qdf['tous_les_mots_uniques'].values)

# Fin de la procédure
print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

##### Creation de notre dictionnaire

In [ ]:
import math

print("   >>> Comptage de l'occurence des mots dans toutes nos phrases : ")
dic_frequence = Counter(vocab)
print(len(dic_frequence))

print("   >>> Comptage de l'occurence des mots uniques dans toutes nos phrases : ")
dic_frequence_pour_doc_freq = Counter(vocab_pour_doc_freq)

print("   >>> Comptage de l'occurence des mots dans toutes nos phrases : ")
frequence_mots = sorted(dic_frequence.items(), key=operator.itemgetter(1), reverse=True)
print(frequence_mots[0:100])
print(frequence_mots[5000:5100])
print(frequence_mots[60000:60100])
print(frequence_mots[-100:])

##### Calcul de l'IDF selon différentes méthodes : 

Différentes formules sont testées :
* dict_idf1 :
* dict_idf2 : 
* dict_idf3 : 

In [ ]:
dict_idf1 = {mot: 1/(10.0+dic_frequence_pour_doc_freq[mot]) for mot in dic_frequence_pour_doc_freq}
dict_idf2 = {mot: math.log(float(len(qdf))/dic_frequence_pour_doc_freq[mot]) for mot in dic_frequence_pour_doc_freq}
dict_idf3 = {mot: math.log((float(len(qdf))-dic_frequence_pour_doc_freq[mot]+0.5) \
                           /(dic_frequence_pour_doc_freq[mot]+0.5)) for mot in dic_frequence_pour_doc_freq}

##### Calcul du TF - IDF selon les différentes formules de l'IDF

Pour déterminer le TF, on calcle simplement l'occurence des mots dans notre document. 
Par exemple, pour le premier document de notre base d'apprentissage (c'est-à-dire la première phrase/question), on a : 

** "what is the step by step guide to invest in share market in india?" ** : 
* 'step': 2, 
* 'in': 2, 
* 'what': 1, 
* 'is': 1,
* 'the': 1, 
* 'by': 1,
* 'guide': 1, 
* 'to': 1,
* 'invest': 1,
* 'share': 1,
* 'market': 1,
* 'india': 1

Ce résultat est obtenu en utilisant la fonction *Counter()*

In [ ]:
def tf_idf1_sac_de_mots(sac_de_mots):
    compt = Counter(sac_de_mots)                       # Nombre de mot dans le sac de mot de la phrase 
    sac_de_mots = list(set(sac_de_mots))               # Lister de manière unique les mots présent dans le sac de mot
    print("\nprint sac_de_mots : \n" , sac_de_mots)
    return {x:(float(compt[x]))*dict_idf1[x] for x in sac_de_mots} 

def tf_idf2_sac_de_mots(sac_de_mots):
    compt = Counter(sac_de_mots)
    sac_de_mots = list(set(sac_de_mots))
    return {x:(float(compt[x]))*dict_idf2[x] for x in sac_de_mots}

def tf_idf3_sac_de_mots(sac_de_mots):
    compt = Counter(sac_de_mots)
    sac_de_mots = list(set(sac_de_mots))
    return {x:(float(compt[x]))*dict_idf3[x] for x in sac_de_mots}

def norm1(dic_tf_idf):
    return sum(list(dic_tf_idf.values()))

def norm2(dic_tf_idf):
    return sum([x*x for x in list(dic_tf_idf.values())])

In [ ]:
# Avec la fonction apply on présente les phrases une a une.
# res=qdf['sac_de_mots'][0:1].apply(lambda x: [tf_idf1_sac_de_mots(y) for y in x]) 

# qdf["tf_idf2"]=qdf['sac_de_mots'].apply(lambda x: [tf_idf2_sac_de_mots(y) for y in x])
# qdf["tf_idf3"]=qdf['sac_de_mots'].apply(lambda x: [tf_idf3_sac_de_mots(y) for y in x])

# print("Nombre de variable : ", len(list(qdf.columns)))
# print("Liste des variables : ", list(qdf.columns))
# qdf.head(5)

In [ ]:
# Avec la fonction apply on présente les phrases une a une.
qdf["tf_idf1"]=qdf['sac_de_mots'].apply(lambda x: [tf_idf1_sac_de_mots(y) for y in x]) 
qdf["tf_idf2"]=qdf['sac_de_mots'].apply(lambda x: [tf_idf2_sac_de_mots(y) for y in x])
qdf["tf_idf3"]=qdf['sac_de_mots'].apply(lambda x: [tf_idf3_sac_de_mots(y) for y in x])

##### Quelques affichages

In [ ]:
qdf.head(5)

### <span style="color:#53c653"> 6.3.3. Methode 2 : Utilisation du module "TfidfVectorizer" de Sklearn

Plus facile a utiliser, moins de risque de se tromper dans les formules mais moins flexibles que précédemment.

Dans ce module, l'IDF est calculé comme suit : 

idf(d,t) = log(n * / df(d,t)) + 1 avec :
* n : le nombre total de document
* df(d,t) : le _document frequency_
* +1 : paramètre de régularisation au cas où le log serait nul.

**Quelques paramètres **: 
* **strip_accents** : Supprimer les accents pendant l'étape de prétraitement
* **ngram_range** : Les limites inférieure et supérieure de la plage de valeurs n pour différents n-grammes à extraire. Toutes les valeurs de n telles que min_n <= n <= max_n seront utilisées
* **min_df** : Lors de la construction du vocabulaire, ignorez les termes qui ont une fréquence de document strictement inférieure au seuil donné
* **max_df** : Lors de la construction du vocabulaire, ignorez les termes qui ont une fréquence de document strictement supérieure au seuil donné (permet d'éliminer notamment les stopwords spécifiques au corpus)
* **max_features** :  Permet de considérer uniquement les x premiers features (ordonné par la fréquence d'occurence dans le corpus), si le paramètre n'est pas mis a "none"
* **use_idf ** : Activer la repondération idf (paramètre booléen)
* **smooth_idf** : permet d'empecher les divisions par 0
* **sublinear_tf ** : Appliquez une mise à l'échelle sur le tf, c'est-à-dire remplacez tf par 1 + log (tf).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy  as sp

start = time.time();

print("Lancement du TF-IDF : \n")

# term-frequency-inverse-document-frequency

tfv = TfidfVectorizer(min_df=2,  
                      max_features=20000, 
                      lowercase=True, 
                      stop_words= 'english',
                      strip_accents='unicode',    
                      ngram_range=(1, 1),        
                      use_idf=1,
                      smooth_idf=1,
                      sublinear_tf=1)
            
# Fit TFIDF - Learn the idf vector (global term weights)
tfv.fit(pd.concat([df_model['question_1'],df_model['question_2']]))

# Transform data - Transform a count matrix to a tf or tf-idf representation
tr1 = tfv.transform(df_model['question_1']) 
tr2 = tfv.transform(df_model['question_2'])

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

print("SHape of tr1", tr1.shape)
print("SHape of tr2", tr2.shape)

**Remarque **: il est possible de lancer le TFIDF sur des bigrams, trigram ... Ngram en modifiant le paramètre **ngram_range**. En effet, si on considère :
* ngram_range = (1, 1) : on réalise le tf-idf sur des unigrams (sac de mot donc ! )
* ngram_range = (1, 2) : le tf idf est réalisé sur des bigrams 
* etc ...

Il est donc possible de réaliser plusieurs TF-IDF et de concatener les résultats.

Exemple de programme python: 
```
# 1. Realisation du TF IDF sur des unigrams
tfidf_word = TfidfVectorizer(analyzer='word',
                             ngram_range=(1, 1)) 
X_word = tfidf_word.fit_transform(X)

# 2. Realisation du TF IDF sur des bigrams
tfidf_2word = TfidfVectorizer(analyzer='char', 
                              ngram_range=(2,2))
X_2word = tfidf_2word.fit_transform(X)

# 3. Concatenation des résultats
X_all_TFIDF = sparse.hstack([X_word, X_2word])
```

### <span style="color:#53c653"> 6.3.4. Analyses des résultats du tfidf

#### Affichage du vocabulaire et du comptage de mot - calculé par TF IDF de scikit learn

In [ ]:
tfv.vocabulary_

#### Recuperation des features créées par le TF-IDF de scikit learn 

* la fonction "get_feature_names()" permet de récupérer la liste de tous les mots du vocabulaire de tf-idf, dans le même ordre que les colonnes de la matrice.

In [ ]:
features = tfv.get_feature_names()
print("Quelques exemples de feature : \n", features[10000:10020])
print("Quelques exemples de feature : \n", features[51:100])

#### Highest score pour une question

Nous créons une fonction qui prend une seule ligne de la matrice tf-idf (correspondant à un document particulier - cad dans notre cas a une phrase), et renvoie les _n_ mots les plus performants.

Comment procède-t-on ? 
* Dans la fonction **"top_tfidf_feats(params)"**: On utilise la fonction _argsort()_ pour  réordonner en ordre décroissant les scores obtenues par le TF IDF puis on sélectionne le premier _top_n_. Nous retournons alors un DataFrame pandas avec les mots eux-mêmes (noms de caractéristiques) et leur score associé.
* Dans la fonction **"top_feats_in_doc(params)"** : le TF IDF de scikit learn produit une **sparse matrix** , qui ne supporte pas toutes les opérations habituelles de matrice ou de tableau. Donc, afin d'appliquer la fonction ci-dessus, nous convertissons cette matrice en un format plus usuel 

In [ ]:
def top_tfidf_feats(row, features, top_n=20):
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats, columns=['features', 'score'])
    return df

def top_feats_in_doc(X, features, row_id, top_n=25):
    row = np.squeeze(X[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

In [ ]:
irow = 1
print("Highest score pour la ligne {} :  \n{}".format(irow, top_feats_in_doc(tr1, features, irow, 5)))

##  <span style="color:#39ac39"> 6.4. Ajout des variables complémentaires

In [ ]:
start = time.time();

print("Ajout des variables complémentaires : ")
# on ajoute toutes les variables numériques que l'on a créée précédemment
df_additional = df_model.drop(["question_1","question_2","is_duplicate"], axis = 1)

df_additional = df_additional.fillna(0)
df_additional.isnull().sum()

y = df_model.is_duplicate.values
X = sp.sparse.hstack([tr1,tr2,np.array(df_additional)])
print("Shape of design matrix : ", X.shape)
print("Shape du vecteur cible : ", y.shape)
# Z = sp.sparse.hstack([ts1,ts2])

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

##  <span style="color:#39ac39"> 6.5. Split en table d'apprentissage et de validation

Pour construire un modèle, on utilise trois tables : 
* La base de <span style="background-color:#ccffcc"> ** "train" ** </span> :
* La base de <span style="background-color:#ccffcc"> ** "validation" ** </span> :
* La base de <span style="background-color:#ccffcc"> ** "test" ** </span> :


![TrainValidTest](https://github.com/sdaymier/NLP/blob/master/TrainValidTest.PNG?raw=true "RandomForest")



In [ ]:
from sklearn.cross_validation import train_test_split

start = time.time();
print("Creation de la base d'apprentissage et de validation : \n")

x_train, x_val, y_train, y_val = train_test_split(X, 
                                                  y, 
                                                  test_size=0.4,
                                                  random_state=42)
print("-------------------------------------------------------- ")
print("Dimension du vecteur cible pour l'apprentissage y_train : ", y_train.shape)
print("Dimension de la design matrix pour l'apprentissage x_train : ", x_train.shape)
print("\n-------------------------------------------------------- ")
print("Dimension du vecteur cible pour la validation y_val : ", y_val.shape)
print("Dimension de la design matrix pour la validation x_val : ", x_val.shape)

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

##  <span style="color:#39ac39"> 6.6. Random Forest

### <span style="color:#53c653"> 6.6.1. Aspect théorique global


Les  <span style="background-color:#ccffcc"> **   forêts aléatoires ** </span> (ou <span style="background-color:#ccffcc"> **   forêts aléatoires ** </span> Random Forest) sont conçues pour améliorer la précision des modèles CART en construisant des plusieurs arbres (c'est-à-dire une forêt!). 
L'inconvénient de l'utilisation de Random Forests est que les modèles deviennent moins faciles à comprendre et moins interprétables, mais ils peuvent améliorer la précision de vos prédictions.


Le schéma suivant synthétise l'algorithme : 
![RandomForest](https://github.com/sdaymier/NLP/blob/master/RandomForest.PNG?raw=true "RandomForest")

### <span style="color:#53c653"> 6.6.2. En pratique avec le module de sklearn

**Note** : temps d'execution - 164.64 secondes avec : 
* n_estimators = 5, 
* min_samples_split = 100

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score, roc_auc_score

start = time.time();
print("RANDOM FOREST : \n")

print("   >>> Setting : ")
rf = RandomForestClassifier(n_estimators = 5, 
                            min_samples_split = 100,  
                            verbose=1)
print("   >>> Fitting : ")
rf.fit(x_train, y_train)

print("Temps d'execution : {:.2f} secondes".format(time.time() - start))

##  <span style="color:#39ac39"> 6.7. Evaluation
### <span style="color:#53c653"> 6.7.1. Quelques metrics !

#### AUC 


In [ ]:
from sklearn import metrics

print("AUC")
print("   >>> AUC sur la base d'apprentissage : ")
x_train_pred =  rf.predict(x_train)
acc = roc_auc_score(y_train, x_train_pred)
print("AUC sur la base de train  : ", acc)

print("\n   >>> AUC sur la base de validation : ")
x_val_pred = rf.predict(x_val)
acc_valid = roc_auc_score(y_val, x_val_pred)
print("AUC sur la base de validation  : ", acc_valid)

#### Accuracy


In [ ]:
print("Accuracy sur la base d'apprentissage : \n", metrics.accuracy_score(y_train,x_train_pred))
print("\nAccuracy sur la base de validation : \n", metrics.accuracy_score(y_val,x_val_pred))

#### Matrice de confusion

In [ ]:
print("Matrice de confusion sur la base d'apprentissage : \n", metrics.confusion_matrix(y_train,x_train_pred))
print("\nMatrice de confusion sur la base de validation : \n", metrics.confusion_matrix(y_val,x_val_pred))

### <span style="color:#53c653"> 6.7.2. Importance des variables

In [ ]:
#This prints the top 10 most important features
res = pd.DataFrame(sorted(zip(rf.feature_importances_, 
           tfv.get_feature_names()), 
       reverse=True))
res.columns = ["Importance","Feature"]
res.head(10)

In [ ]:
import seaborn as sns
data_to_plot = res[0:10]
plt.figure(figsize = (8,5))
sns.barplot(x = data_to_plot.Importance, y = data_to_plot.Feature, orient = 'h')
plt.title('Importance des variables - Random Forest Result', fontsize = 15)